In [1]:
#Mount drive to access files in gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import string
import re
import keras
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model


In [3]:
# reading semval datasets
df_train = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/dev.csv")
df_gold = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/gold-test-27446.csv")

In [4]:
# text preprocessing
%%time
def text_preprocessing(text):
  text = text.lower()
  text = re.sub(r"http\S+", " ", text)
  text = re.sub(' +', ' ', text)
  
  return text

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [5]:
#applying text preprocessing on all dataset

# train data
df_train_1 = df_train.copy()
df_train_1['text'] = df_train_1['text'].apply(text_preprocessing)

#test data
df_test_1 = df_test.copy()
df_test_1['text'] = df_test_1['text'].apply(text_preprocessing)

#gold data
df_gold_1 = df_gold.copy()
df_gold_1['text'] = df_gold_1['text'].apply(text_preprocessing)

In [6]:
# intializing tokenizer and fitting on train, test and dev data and padding to each sequences

vocabulary_size = 20000
maxlen = 200

tokenizer = Tokenizer(num_words= vocabulary_size)

#creating vocabulary on train data
tokenizer.fit_on_texts(df_train_1['text'])

#generating sequence of tokens on train, text, gold data
train_sequences = tokenizer.texts_to_sequences(df_train_1['text'])
test_sequences = tokenizer.texts_to_sequences(df_test_1['text'])
gold_sequences = tokenizer.texts_to_sequences(df_gold_1['text'])

#padding the sequences
train_pad = pad_sequences(train_sequences, maxlen=maxlen)
test_pad = pad_sequences(test_sequences, maxlen=maxlen)
gold_pad = pad_sequences(gold_sequences, maxlen=maxlen)

In [7]:
# reading glove embedding and creating weight matrix 
def read_glove_embedding(vocabulary_size, df, glove_path, embedding_size, tokenizer):

  embeddings_index = dict()

  f = open(glove_path)
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  print('Loaded %s word vectors.' % len(embeddings_index))

  embedding_matrix = np.zeros((vocabulary_size, embedding_size))
  for word, index in tokenizer.word_index.items():
      if index > vocabulary_size - 1:
          break
      else:
          embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

  return embedding_matrix


In [8]:
#create embedding for train data
vocabulary_size = 20000
df_new = df_train_1
glove_path = '/content/gdrive/MyDrive/Lakehead_NLP_Project/glove.6B.300d.txt'
embedding_size = 300

embedding_matrix = read_glove_embedding(vocabulary_size, df_new, glove_path,  embedding_size, tokenizer)

Loaded 400000 word vectors.


In [9]:
# defining metrices to evaluate the model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
# constructing model

embedding_input_shape = 300
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, embedding_input_shape, input_length=150, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy',f1_m, precision_m, recall_m])


In [11]:
%%time
#fitting model on train data
model_glove.fit(train_pad, np.array(df_train_1['is_humor']), validation_data=(test_pad, np.asarray(df_test_1['is_humor'])), epochs = 10, batch_size=128)

Epoch 1/10
63/63 [==============================] - 53s 52ms/step - loss: 0.5806 - binary_accuracy: 0.6727 - f1_m: 0.7347 - precision_m: 0.6499 - recall_m: 0.8616 - val_loss: 0.4406 - val_binary_accuracy: 0.7840 - val_f1_m: 0.8378 - val_precision_m: 0.7883 - val_recall_m: 0.8954
Epoch 2/10
63/63 [==============================] - 2s 38ms/step - loss: 0.3494 - binary_accuracy: 0.8400 - f1_m: 0.8740 - precision_m: 0.8578 - recall_m: 0.8960 - val_loss: 0.4229 - val_binary_accuracy: 0.8040 - val_f1_m: 0.8531 - val_precision_m: 0.7995 - val_recall_m: 0.9162
Epoch 3/10
63/63 [==============================] - 2s 35ms/step - loss: 0.2734 - binary_accuracy: 0.8861 - f1_m: 0.9071 - precision_m: 0.9067 - recall_m: 0.9121 - val_loss: 0.3825 - val_binary_accuracy: 0.8110 - val_f1_m: 0.8511 - val_precision_m: 0.8370 - val_recall_m: 0.8680
Epoch 4/10
63/63 [==============================] - 2s 35ms/step - loss: 0.2153 - binary_accuracy: 0.9134 - f1_m: 0.9283 - precision_m: 0.9319 - recall_m: 0.9271 

In [14]:
# evaluating model on gold data
gold_lstm_results = model_glove.evaluate(gold_pad, np.asarray(df_gold_1['is_humor']), verbose=0, batch_size=128)

print(f'Gold Data accuracy: {gold_lstm_results[1]*100:0.2f}')
print(f'Gold Data F1 Score: {gold_lstm_results[2]}')

Gold Data accuracy: 81.80
Gold Data F1 Score: 0.8452799916267395
